In [1]:
%matplotlib inline
import numpy as np
import numpy.random as rand
import matplotlib.pyplot as plt

## Generate multi-arm machines
* user-input: arm no.
* return: loss

In [2]:
rand.rand(3)

array([ 0.41124664,  0.8806728 ,  0.97419391])

In [3]:
class Class_Dict:
    def __iter__(self):
        for attr in dir(Class_Dict):
            if not attr.startswith("__"):
                yield attr

config = Class_Dict()
config.n_arms = 4
config.initial_arm_status = np.zeros(config.n_arms)

class Arms:
    def __init__(self, config):
        self.n_arms = config.n_arms
        self.hidden_status = rand.rand(self.n_arms)
        
        
    def pulled(self, i_arm):
        p = self.hidden_status[i_arm]
        return rand.binomial(1, p)
        

arms = Arms(config)

## Human-playable multi-arm bandit game

### Introduce general bandit

In [4]:
class Bandit:
    def __init__(self, config):
        self.n_arms = config.n_arms
        self.reward_history = []
        self.arm_history = []
    
    # abstract
    def choose(self):
        pass
    
    # abstract
    def learn(self):
        pass
    
    def pull_and_learn(self, arms):
        i_arm = self.choose()
        reward = arms.pulled(i_arm)
        
        self.arm_history.append(i_arm)
        self.reward_history.append(reward)
    
        self.learn(i_arm, reward)
        
    def plot_history(self):
        plt.plot(np.cumsum(self.reward_history))
        plt.show()
        plt.scatter(range(len(self.arm_history)), self.arm_history)
        plt.show()

### Define Human-player

In [5]:
class Player(Bandit):
    def __init__(self, config):
        Bandit.__init__(self, config)
        
    def choose(self):
        return int(input("Enter a number from 1 to " + str(self.n_arms) + ": "))
    
    def learn(self, i_arm, reward):
        print("Your choice is arm: ", i_arm)
        print("Your reward is: ", reward)

    def start(self, arms):
        rounds = int(input("Enter the number of rounds you want to play: "))
        print("Choose the arm you think that will give you more reward", )
        print("from a total number of " + str(self.n_arms) + " arms", )
        for _ in range(rounds):
            self.pull_and_learn(arms)
        
        
player = Player(config)
player.start(arms)

Enter the number of rounds you want to play: 5
Choose the arm you think that will give you more reward
from a total number of 4 arms
Enter a number from 1 to 4: 1
Your choice is arm:  1
Your reward is:  1
Enter a number from 1 to 4: 1
Your choice is arm:  1
Your reward is:  0
Enter a number from 1 to 4: 2
Your choice is arm:  2
Your reward is:  1
Enter a number from 1 to 4: 2
Your choice is arm:  2
Your reward is:  1
Enter a number from 1 to 4: 2
Your choice is arm:  2
Your reward is:  1


## Define the Thompson bandit

In [8]:
class Thompson(Bandit):
    def __init__(self, config):
        Bandit.__init__(self, config)
        
        # Define the prior of the Beta distribution
        self.a = np.ones(self.n_arms)
        self.b = np.ones(self.n_arms)
        
    def choose(self):
        guess = np.zeros(self.n_arms)
        for i_arm in range(self.n_arms):
            guess[i_arm] = rand.beta(self.a[i_arm], self.b[i_arm])
        return np.argmax(guess)
    
    def learn(self, i_arm, reward):
        # According to beta posterior update by binomial(n=1) data 
        self.a[i_arm] += reward
        self.b[i_arm] += 1 - reward

[ 0.51801404  0.70352451  0.71283947  0.19968548]


/usr/lib/python3/dist-packages/matplotlib/collections.py:571: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


In [9]:
th = Thompson(config)
print(arms.hidden_status)
for _ in range(100):
    th.pull_and_learn(arms)

th.plot_history()